# Accessing cloud-hosted image data (practical)

***

**Global Bioimaging Online Course**

*Image Data: data management, standard image file format for sharing big image data in the cloud, and image data repositories*

*Day 3: Friday, 29 January 2021 – Standard image file format for sharing big image data in the cloud*

***

A PDF version of the presentation is available at 
https://downloads.openmicroscopy.org/presentations/2021/GBI 


## Summary
0. Software versions
1. Notebook reminders
2. Choosing our data & downloading from S3
3. Converting your data to OME-NGFF
4. Publishing your data with S3
5. Extras (time-permitting)

***

## 0. Software versions used for this workshop:


### 0.1 Running now

* jupyter                   1.0.0 ([external guide](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/))
* conda 4.8.4 ([external guide](https://docs.conda.io/projects/conda/en/latest/user-guide/index.html))

### 0.2 Core tools
* **bioformats2raw            0.2.5** ([install locally](https://github.com/glencoesoftware/bioformats2raw/releases/latest); requires Java)
* **minio-client                     2020.11.17** ([install locally](https://docs.min.io/docs/minio-client-complete-guide.html))

### 0.3 Other

   * awscli                    1.18.219
   * dask                      2021.1.0
   * fsspec                    0.8.5
   * napari                    0.4.3
   * numpy                     1.19.5
   * ome-zarr                  0.0.17
   * omero-cli-zarr            0.0.9
   * omero-py                  5.8.2
   * openjdk                   11.0.8
   * tifffile                  2021.1.14
   * zarr                      2.6.1

***

## 1. Notebook reminders

This notebook is somewhat unusual in that we use a lot of command-line tools. Each of the lines beginning with a exclamation mark (`!`) is run in a terminal.


In [17]:
!pwd

/opt/NGFF-GBI-2021-Workshop


In [18]:
!ls binder/

apt.txt         environment.yml


In [9]:
!cat binder/environment.yml

channels:
  - ome
  - conda-forge
  - joshmoore
  - defaults
dependencies:
  - python == 3.9.0
  - napari
  - ipython
  - wheel
  - requests
  - s3fs
  - scikit-image
  - scipy
  - xarray
  - zarr >= 2.4.0
  - bioformats2raw
  - omero-py
  - go-mc
  - pip
  - pip:
      - PyQT5==5.13.1
      - ome-zarr
      - awscli
      - omero-cli-zarr


You can perform those actions on your own system _without_ a Jupyter notebook. For that, you will need to install the correct tools like `bioformats2raw` or run everything via `repo2docker`.

See instructions under https://github.com/joshmoore/NGFF-GBI-2021-Workshop/blob/main/README.md
 
*** 

## 2. Data from S3
We're going to start off by looking at some images you will likely have seen during the OMERO or IDR sessions.

**Our goal is to share these *without* using an OMERO.**

<table>
    <tr>
        <td>
            <img alt="idr0062 thumbnails" src="images/training-1.png" style="height:150px"/>
        </td>
        <td>
            <img alt="idr0062 thumbnails" src="images/training-2.png" style="height:150px"/>
        </td>
        <td>
            <img alt="idr0023 3D screenshot" src="images/training-3.png" style="height:150px"/>
        </td>
    </tr>
</table>
    
The left two images are from  the ilastik plugin guide presented by Petr: https://omero-guides.readthedocs.io/en/latest/ilastik/docs/ilastik_fiji.html

They are available in the "idr0062" project on the workshop server: https://workshop.openmicroscopy.org/webclient/?show=project-1952

The original dataset can be found in IDR study idr0062 by Blin _et al._: https://idr.openmicroscopy.org/webclient/?show=project-801

The image on the right is from idr0023 by Szymborska _et al_: http://idr.openmicroscopy.org/webclient/?show=project-52 and is **much** smaller.


## 2.1 Minio client

There are a number of different types of cloud storage and there are a number of tools that you can use to access your cloud storage, but here we're going to focus on a single one `mc`.

`mc` is provided by the minio project and is described as "a modern alternative to UNIX commands like ls, cat, cp, mirror, diff, find etc." The quickstart guide can be found under https://docs.minio.io/docs/minio-client-quickstart-guide.html For our purposes we'll focus on how to use it to upload and manage data in S3.

In [15]:
!mc

NAME:
  mc - MinIO Client for cloud storage and filesystems.

USAGE:
  mc [FLAGS] COMMAND [COMMAND FLAGS | -h] [ARGUMENTS...]

COMMANDS:
  alias      set, remove and list aliases in configuration file
  ls         list buckets and objects
  mb         make a bucket
  rb         remove a bucket
  cp         copy objects
  mirror     synchronize object(s) to a remote site
  cat        display object contents
  head       display first 'n' lines of an object
  pipe       stream STDIN to an object
  share      generate URL for temporary access to an object
  find       search for objects
  sql        run sql queries on objects
  stat       show object metadata
  mv         move objects
  tree       list buckets and objects in a tree format
  du         summarize disk usage recursively
  retention  set retention for object(s)
  legalhold  manage legal hold for object(s)
  diff       list differences in object name, size, and date between two buckets
  rm         remove objects
  version    

## 2.2 Connections

The minio project provides a safe space for you to learn about S3: https://play.minio.io:9000/minio/ Here we've used the `mc` command to find the access information:

 * "AccessKey" is basically a user name.
 * "SecretKey" is basically a password. 
 * The URL is our "endpoint", which differentiates it from the S3 servers provided by Amazon.

You can log in to the webpage and explore what the many other users have upload at https://play.minio.io:9000/minio/

The other two important concepts are:
 * "buckets" which is roughly like a shared namespace with permissions
 * and "keys" which will get to in a second.

In [16]:
!mc config host list play

play
  URL       : https://play.min.io
  AccessKey : Q3AM3UQ867SPQQA43P2F
  SecretKey : zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG
  API       : S3v4
  Path      : auto



## 2.3 Adding your own

But EMBL has kindly provided us a bucket for this session which we need to connect to:

In [16]:
!mc config host add gbi https://s3.embl.de bioim-user PLEASE_FIX_ME

mc: <ERROR> Unable to initialize new alias from the provided credentials. The request signature we calculated does not match the signature you provided. Check your key and signing method.


In [19]:
!mc ls gbi/bioim

[2021-01-21 18:26:48 CET]  32KiB s3-browser.html
[2021-01-28 10:27:43 CET]     0B idr0023/
[2021-01-28 10:27:43 CET]     0B idr0062-tiffs/
[2021-01-28 10:27:43 CET]     0B idr0062-zarrs/


In [20]:
!mc ls gbi/bioim/idr0062-tiffs/

[2021-01-21 18:07:25 CET]  68MiB B1_C1.tif
[2021-01-21 18:07:25 CET]  67MiB B1_C1_Manual.tif
[2021-01-21 18:07:25 CET]  66MiB B1_C2.tif
[2021-01-21 18:07:25 CET]  66MiB B1_C2_Manual.tif
[2021-01-21 18:07:25 CET] 120MiB B2_C1.tif
[2021-01-21 18:07:25 CET] 120MiB B2_C1_Manual.tif
[2021-01-21 18:07:25 CET]  42MiB B2_C2.tif
[2021-01-21 18:07:25 CET]  42MiB B2_C2_Manual.tif
[2021-01-21 18:07:29 CET]  84MiB B3.tif
[2021-01-21 18:07:29 CET]  83MiB B3_Manual.tif
[2021-01-21 18:07:29 CET]  49MiB B4_C1.tif
[2021-01-21 18:07:29 CET]  49MiB B4_C1_Manual.tif
[2021-01-21 18:07:29 CET]  79MiB B4_C2.tif
[2021-01-21 18:07:29 CET]  78MiB B4_C2_Manual.tif
[2021-01-21 18:07:29 CET]  53MiB B4_C3.tif
[2021-01-21 18:07:29 CET]  52MiB B4_C3_Manual.tif
[2021-01-21 18:07:30 CET]  47MiB B5_C1.tif
[2021-01-21 18:07:30 CET]  47MiB B5_C1_Manual.tif
[2021-01-21 18:07:31 CET]  44MiB B5_C2.tif
[2021-01-21 18:07:30 CET]  44MiB B5_C2_Manual.tif
[2021-01-21 18:07:33 CET]  44MiB B5_C3.tif
[2021-01-21 18:07:33 CET]  44MiB 

In [21]:
!mc ls gbi/bioim/idr0062-zarrs/

[2021-01-28 10:27:51 CET]     0B 6001237.zarr/
[2021-01-28 10:27:51 CET]     0B 6001238.zarr/
[2021-01-28 10:27:51 CET]     0B 6001239.zarr/
[2021-01-28 10:27:51 CET]     0B 6001240.zarr/
[2021-01-28 10:27:51 CET]     0B 6001241.zarr/
[2021-01-28 10:27:51 CET]     0B 6001242.zarr/
[2021-01-28 10:27:51 CET]     0B 6001243.zarr/
[2021-01-28 10:27:51 CET]     0B 6001244.zarr/
[2021-01-28 10:27:51 CET]     0B 6001245.zarr/
[2021-01-28 10:27:51 CET]     0B 6001246.zarr/
[2021-01-28 10:27:51 CET]     0B 6001247.zarr/
[2021-01-28 10:27:51 CET]     0B 6001248.zarr/
[2021-01-28 10:27:51 CET]     0B 6001249.zarr/
[2021-01-28 10:27:51 CET]     0B 6001250.zarr/
[2021-01-28 10:27:51 CET]     0B 6001251.zarr/
[2021-01-28 10:27:51 CET]     0B 6001252.zarr/
[2021-01-28 10:27:51 CET]     0B 6001253.zarr/
[2021-01-28 10:27:51 CET]     0B 6001254.zarr/
[2021-01-28 10:27:51 CET]     0B 6001255.zarr/
[2021-01-28 10:27:51 CET]     0B 6001256.zarr/
[2021-01-28 10:27:51 CET]     0B 6001257.zarr/
[2021-01-28 1

In [23]:
!mc ls gbi/bioim/idr0023/

[2021-01-28 10:22:33 CET]    44B README.txt
[2021-01-28 10:21:53 CET] 2.0MiB trans_norm.tif
[2021-01-28 10:28:20 CET]     0B trans_norm.ome.zarr/


## 2.3 Your first download

Now you can see list the contents of the bucket and see that it's there. Note that since other people are working on the same bucket, it may look slightly different each time you run this command.

In [24]:
!mc cp gbi/bioim/idr0023/trans_norm.tif /tmp/

..._norm.tif:  2.04 MiB / 2.04 MiB  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  1.25 MiB/s 1s

In [25]:
!ls -ltrah /tmp/trans_norm.tif

-rw-r--r--  1 jamoore  wheel   2.0M Jan 28 10:28 /tmp/trans_norm.tif


***

# 3 Converting your data to OME-NGFF

The two basic commands are `bioformats2raw` and `raw2ometiff`. Together they provide a pipeline to scalably convert large images into OME-TIFF. The primary caveat is that they require **twice** the storage for the conversion.


# 3.1 Conversion tools

https://forum.image.sc/t/converting-whole-slide-images-to-ome-tiff-a-new-workflow/32110/4

<img src="blog-2019-12-converting-whole-slide-images.jpg" style="height:200px" />



In [28]:
!bioformats2raw

Missing required parameters: <inputPath>, <outputPath>
Usage: <main class> [--debug] [--version] [--extra-readers[=<extraReaders>[,
                    <extraReaders>...]]]...
                    [--additional-scale-format-string-args=<additionalScaleForma
                    tStringArgsCsv>] [-c=<compressionType>]
                    [--compression-parameter=<compressionParameter>]
                    [--dimension-order=<dimensionOrder>]
                    [--file_type=<fileType>] [-h=<tileHeight>]
                    [--max_cached_tiles=<maxCachedTiles>]
                    [--max_workers=<maxWorkers>]
                    [--memo-directory=<memoDirectory>]
                    [--pyramid-name=<pyramidName>] [-r=<pyramidResolutions>]
                    [--scale-format-string=<scaleFormatString>]
                    [-w=<tileWidth>] <inputPath> <outputPath>
      <inputPath>    file to convert
      <outputPath>   path to the output pyramid directory
      --additional-scale-format-st

## Required OME-Zarr options
Two of the options are currently necessary to produce OME-Zarr data:
```
      --file_type=<fileType>
                     Tile file extension: n5, zarr (default: n5) [Can break
                       compatibility with raw2ometiff]
```
and
```
     --dimension-order=<dimensionOrder>
                     Override the input file dimension order in the output file
                       [Can break compatibility with raw2ometiff] (XYZCT,
                       XYZTC, XYCTZ, XYCZT, XYTCZ, XYTZC)
```

`--file_type` which produces Zarr output rather than N5 as the intermediate format. If we additionally pass the `--dimension-order` argument, then the intermediate result can be used directly by the ome-zarr library.

In [29]:
%%time
!bioformats2raw /tmp/trans_norm.tif /tmp/trans_norm_out --file_type=zarr --dimension-order=XYZCT

2021-01-28 10:29:47,197 [main] INFO  c.g.bioformats2raw.Converter - Output will be incompatible with raw2ometiff (pyramidName: data.zarr, scaleFormatString: %d/%d)
2021-01-28 10:29:47,887 [main] ERROR loci.formats.ImageReader -  *** One or more readers is misbehaving. See the debug output for more information. e.g.:
     loci.formats.in.APLReader@42f8285e -> java.lang.NullPointerException('null') ***
2021-01-28 10:29:47,934 [main] INFO  loci.formats.ImageReader - TiffDelegateReader initializing /tmp/trans_norm.tif
2021-01-28 10:29:47,936 [main] INFO  loci.formats.in.MinimalTiffReader - Reading IFDs
2021-01-28 10:29:47,971 [main] INFO  loci.formats.in.MinimalTiffReader - Populating metadata
2021-01-28 10:29:47,996 [main] INFO  loci.formats.in.TiffReader - Checking comment style
2021-01-28 10:29:47,997 [main] INFO  loci.formats.in.BaseTiffReader - Populating OME metadata
2021-01-28 10:29:48,034 [main] INFO  loci.formats.in.MinimalTiffReader - Reading IFDs
2021-01-28 10:29:48,055 [main] I

2021-01-28 10:29:48,768 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 33, 0, 0] to /0/0
2021-01-28 10:29:48,768 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=33 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:48,769 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 34, 0, 0] to /0/0
2021-01-28 10:29:48,770 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 32/571
2021-01-28 10:29:48,770 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1611826188769] time[0] tag[getTile]
2021-01-28 10:29:48,770 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 34, 0, 0] to /0/0
2021-01-28 10:29:48,770 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=34 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:48,771 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter 

2021-01-28 10:29:48,819 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 60, 0, 0] to /0/0
2021-01-28 10:29:48,819 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 58/571
2021-01-28 10:29:48,819 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1611826188819] time[0] tag[getTile]
2021-01-28 10:29:48,820 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 60, 0, 0] to /0/0
2021-01-28 10:29:48,820 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=60 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:48,821 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 61, 0, 0] to /0/0
2021-01-28 10:29:48,821 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 59/571
2021-01-28 10:29:48,821 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1611826188821] time[0] tag[getTile]
2021-01-2

2021-01-28 10:29:48,870 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 88, 0, 0] to /0/0
2021-01-28 10:29:48,871 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 86/571
2021-01-28 10:29:48,871 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1611826188870] time[0] tag[getTile]
2021-01-28 10:29:48,873 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 88, 0, 0] to /0/0
2021-01-28 10:29:48,873 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=88 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:48,874 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 89, 0, 0] to /0/0
2021-01-28 10:29:48,874 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 87/571
2021-01-28 10:29:48,874 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1611826188874] time[0] tag[getTile]
2021-01-2

2021-01-28 10:29:48,921 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 123, 0, 0] to /0/0
2021-01-28 10:29:48,921 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - tile read complete 121/571
2021-01-28 10:29:48,921 [pool-1-thread-1] INFO  org.perf4j.TimingLogger - start[1611826188921] time[0] tag[getTile]
2021-01-28 10:29:48,921 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 123, 0, 0] to /0/0
2021-01-28 10:29:48,921 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=123 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:48,922 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 124, 0, 0] to /0/0
2021-01-28 10:29:48,923 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - tile read complete 122/571
2021-01-28 10:29:48,923 [pool-1-thread-1] INFO  org.perf4j.TimingLogger - start[1611826188922] time[0] tag[getTile]
202

...

2021-01-28 10:29:49,175 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 352, 0, 0] to /0/0
2021-01-28 10:29:49,176 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 352/571
2021-01-28 10:29:49,176 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1611826189175] time[0] tag[getTile]
2021-01-28 10:29:49,176 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 352, 0, 0] to /0/0
2021-01-28 10:29:49,176 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=352 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:49,177 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 353, 0, 0] to /0/0
2021-01-28 10:29:49,177 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 353/571
2021-01-28 10:29:49,177 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1611826189177] time[0] tag[getTile]
202

2021-01-28 10:29:49,226 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 407, 0, 0] to /0/0
2021-01-28 10:29:49,226 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=407 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:49,227 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 408, 0, 0] to /0/0
2021-01-28 10:29:49,227 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - tile read complete 407/571
2021-01-28 10:29:49,227 [pool-1-thread-3] INFO  org.perf4j.TimingLogger - start[1611826189227] time[0] tag[getTile]
2021-01-28 10:29:49,227 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 408, 0, 0] to /0/0
2021-01-28 10:29:49,227 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=408 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:49,228 [pool-1-thread-3] INFO  c.g.bioformats2raw.Conv

2021-01-28 10:29:49,277 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - tile read complete 460/571
2021-01-28 10:29:49,277 [pool-1-thread-3] INFO  org.perf4j.TimingLogger - start[1611826189277] time[0] tag[getTile]
2021-01-28 10:29:49,278 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 461, 0, 0] to /0/0
2021-01-28 10:29:49,278 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=461 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:49,278 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 462, 0, 0] to /0/0
2021-01-28 10:29:49,278 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - tile read complete 461/571
2021-01-28 10:29:49,278 [pool-1-thread-3] INFO  org.perf4j.TimingLogger - start[1611826189278] time[0] tag[getTile]
2021-01-28 10:29:49,279 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 462, 0, 0] to /0/0
2021-01-2

2021-01-28 10:29:49,328 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - tile read complete 525/571
2021-01-28 10:29:49,328 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1611826189328] time[0] tag[getTile]
2021-01-28 10:29:49,329 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 526, 0, 0] to /0/0
2021-01-28 10:29:49,329 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=526 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:49,329 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 527, 0, 0] to /0/0
2021-01-28 10:29:49,329 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - tile read complete 526/571
2021-01-28 10:29:49,329 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1611826189329] time[0] tag[getTile]
2021-01-28 10:29:49,330 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 527, 0, 0] to /0/0
2021-01-2

2021-01-28 10:29:49,418 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 501, 0, 0] to /0/0
2021-01-28 10:29:49,418 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=501 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:49,421 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 516, 0, 0] to /0/0
2021-01-28 10:29:49,422 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - tile read complete 570/571
2021-01-28 10:29:49,422 [pool-1-thread-1] INFO  org.perf4j.TimingLogger - start[1611826189421] time[0] tag[getTile]
2021-01-28 10:29:49,422 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 516, 0, 0] to /0/0
2021-01-28 10:29:49,422 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=516 xx=0 yy=0 width=30 height=30
2021-01-28 10:29:49,422 [pool-1-thread-2] INFO  c.g.bioformats2raw.Conv

In [33]:
!ls /tmp/trans_norm_out

METADATA.ome.xml data.zarr


In [34]:
!find /tmp/trans_norm_out/data.zarr -name ".z*"

/tmp/trans_norm_out/data.zarr/.zattrs
/tmp/trans_norm_out/data.zarr/.zgroup
/tmp/trans_norm_out/data.zarr/0/.zattrs
/tmp/trans_norm_out/data.zarr/0/.zgroup
/tmp/trans_norm_out/data.zarr/0/0/.zarray


In [37]:
!ome_zarr info /tmp/trans_norm_out/data.zarr/0

/private/tmp/trans_norm_out/data.zarr/0 [zgroup]
 - metadata
   - Multiscales
 - data
   - (1, 1, 571, 30, 30)


***

## 4. Publishing your data with S3

You can then move the generated output to S3. Note: one of the most frequent mistakes here is the slash (`/`) at the end of the commands.x

In [39]:
!time mc cp --recursive /tmp/trans_norm_out/data.zarr/0/ gbi/bioim/josh/my_trans_norm.ome.zarr/

....0.99.0.0:  772.90 KiB / 772.90 KiB  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  50.70 KiB/s 15s
real	0m15.688s
user	0m0.582s
sys	0m0.521s


In [42]:
!mc cat gbi/bioim/josh/my_trans_norm.ome.zarr/.zattrs

{"multiscales":[{"datasets":[{"path":"0"}],"version":"0.1"}]}

You can see your image under http://hms-dbmi.github.io/vizarr?source=https://s3.embl.de/bioim/josh/my_trans_norm.ome.zarr

In [44]:
from IPython.display import IFrame
IFrame("https://hms-dbmi.github.io/vizarr/?source=https://s3.embl.de/bioim/josh/my_trans_norm.ome.zarr/", width=700, height=350)

## 5. Extras (time-permitting

## 5.1 A larger example (idr0062)

In [13]:
from IPython.display import IFrame
IFrame("https://hms-dbmi.github.io/vizarr/?source=https://s3.embl.de/bioim/idr0062-zarrs/6001240.zarr/", width=700, height=350)

# 5.2 Renaming

Another important distinction to filesystems is that though it looks like hello is in a directory, you should really think of the entire string after the bucket just as a "key".

In [49]:
!mc mv --recursive gbi/bioim/josh/my_trans_norm.ome.zarr/ gbi/bioim/josh/public_trans_norm.ome.zarr

....0.99.0.0:  772.90 KiB / 772.90 KiB  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  69.10 KiB/s 11s

## 5.3 omero-cli-zarr

Loading the image from: https://workshop.openmicroscopy.org/webclient/img_detail/55500/?dataset=6210

In [51]:
# Python block for capturing a password
import getpass
mypass = getpass.getpass()

········


In [54]:
!omero -s trainer-1@workshop.openmicroscopy.org -w $mypass zarr export Image:55500

Using session for trainer-1@workshop.openmicroscopy.org:4064. Idle timeout: 10 min. Current group: Lab1
Exporting to 55500.zarr
Finished.


In [55]:
!find 55500.zarr -name ".z*"

55500.zarr/.zattrs
55500.zarr/.zgroup
55500.zarr/0/.zarray
55500.zarr/1/.zarray
55500.zarr/2/.zarray


## 5.4 Other resources

<table>
    <tr>
        <td>
            <a href="https://downloads.openmicroscopy.org/presentations/2020/Dundee/Workshops/NGFF/zarr_diagram/">
<img src="images/resources-1.png" alt="Screenshot of the Zarr diagram from OME2020" style="height:200px"/>
            </a>
        </td>
        <td>
<a href="https://downloads.openmicroscopy.org/presentations/2020/Dundee/Workshops/NGFF/zarr_diagram/">Diagram for how data moves</a>
        </td>
    </tr>
    <tr>
        <td>
      <a href="https://blog.openmicroscopy.org/file-formats/community/2020/11/04/zarr-data/">      
<img src="images/resources-2.png" alt="Screenshot of the Zarr diagram from OME2020" style="height:200px"/>
            </a>
        </td>
        <td>
<a href="https://blog.openmicroscopy.org/file-formats/community/2020/11/04/zarr-data/">Blog post for an easy way to publish OME-Zarr files</a>
        </td>
    </tr>
</table>    

### License
Copyright (C) 2021 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.